In [146]:
import pandas as pd 
import os

In [147]:
# Define Paths
current_working_direct = r"C:\Users\Diego\GenX\GenX.jl-main\example_systems\1_three_zones"
path_results_to_enforce = r"C:\Users\Diego\GenX\GenX.jl-main\example_systems\1_three_zones\results_19_01_6.43pm_16_epsilon_version_few_missing\_CO2=0_MinCapReq=1_COMMIT=0_epsilon=0.02559659_V1"
source_name = "resource_enforce_test3"

In [148]:
import os
import shutil
import pandas as pd

def create_resource_policies_enforce_cap(current_working_direct, path_results_to_enforce, source_name):
    path_capacity_enforce = os.path.join(path_results_to_enforce, "capacity.csv")
    capacity_to_enforce = pd.read_csv(path_capacity_enforce)
    capacity_to_enforce.drop(len(capacity_to_enforce)-1, inplace=True)

    # Define folders to copy
    folder_resource_to_copy = "resources"
    # folder_policy_to_copy = "policies"

    source_folder = os.path.join(current_working_direct, folder_resource_to_copy)
    # policies_folder = os.path.join(current_working_direct, folder_policy_to_copy)

    # Define new target folders
    new_source_folder = os.path.join(current_working_direct, source_name, folder_resource_to_copy + "_" + os.path.basename(path_results_to_enforce))
    # new_policies_folder = os.path.join(current_working_direct, source_name, folder_policy_to_copy + "_" + os.path.basename(path_results_to_enforce))

    # Create new directories and copy resources/policies if they don't exist
    os.makedirs(new_source_folder, exist_ok=True)
    shutil.copytree(source_folder, new_source_folder, dirs_exist_ok=True)

    # os.makedirs(new_policies_folder, exist_ok=True)
    # shutil.copytree(policies_folder, new_policies_folder, dirs_exist_ok=True)
    # Load policy and resource files
    # path_min_cap = os.path.join(current_working_direct, "policies", "Minimum_capacity_requirement.csv")
    # min_cap_no_enforce = pd.read_csv(path_min_cap)

    # path_res_min_cap = os.path.join(current_working_direct, "resources", "policy_assignments", "Resource_minimum_capacity_requirement.csv")
    # res_min_cap_no_enforce = pd.read_csv(path_res_min_cap)
    dict_tech_to_df = pd.DataFrame(columns=["technology", "df_name"])

    path_thermal = os.path.join(new_source_folder,  "Thermal.csv")
    if os.path.exists(path_thermal):
        df_thermal = pd.read_csv(path_thermal)

        dict_tech_to_df = pd.concat([ dict_tech_to_df, pd.DataFrame({
                "technology" : df_thermal["Resource"],
                "df_name" : "df_thermal"
            })])

    path_hydro = os.path.join(new_source_folder, "Hydro.csv")
    if os.path.exists(path_hydro):
        df_hydro = pd.read_csv(path_hydro)

        dict_tech_to_df = pd.concat([ dict_tech_to_df, pd.DataFrame({
                "technology" : df_hydro["Resource"],
                "df_name" : "df_hydro"
            })])

    path_vre = os.path.join(new_source_folder, "Vre.csv")
    if os.path.exists(path_vre):
        df_vre = pd.read_csv(path_vre)

        dict_tech_to_df = pd.concat([ dict_tech_to_df, pd.DataFrame({
                "technology" : df_vre["Resource"],
                "df_name" : "df_vre"
            })])

    path_electrolyzer = os.path.join(new_source_folder, "Electrolyzer.csv")
    if os.path.exists(path_electrolyzer):
        df_electrolyzer = pd.read_csv(path_electrolyzer)

        dict_tech_to_df = pd.concat([ dict_tech_to_df, pd.DataFrame({
                "technology" : df_electrolyzer["Resource"],
                "df_name" : "df_electrolyzer"
            })])

    path_storage = os.path.join(new_source_folder, "Storage.csv")
    if os.path.exists(path_storage):
        df_storage = pd.read_csv(path_storage)

        dict_tech_to_df = pd.concat([ dict_tech_to_df, pd.DataFrame({
                "technology" : df_storage["Resource"],
                "df_name" : "df_storage"
            })])

    path_Vre_stor = os.path.join(new_source_folder, "Vre_stor.csv")
    if os.path.exists(path_Vre_stor):
        df_vre_storage = pd.read_csv(path_Vre_stor)

        dict_tech_to_df = pd.concat([ dict_tech_to_df, pd.DataFrame({
                "technology" : df_vre_storage["Resource"],
                "df_name" : "df_vre_storage"
            })])
        
    dataframes_dict = {
        df_name: globals().get(df_name) for df_name in 
        ["df_thermal", "df_hydro", "df_vre", "df_electrolyzer", "df_storage", "df_vre_storage"]
        if globals().get(df_name) is not None  # Ensures only existing DataFrames are included
    }

    capacity_to_enforce
    for key, df_resource in dataframes_dict.items():
        for index, row in df_resource.iterrows():
            cap_MW = capacity_to_enforce[capacity_to_enforce["Resource"] == row["Resource"]]["EndCap"]
            cap_MW = cap_MW.iloc[0] if not cap_MW.empty else None  # Extract scalar or set to None
            df_resource.loc[index, "Min_Cap_MW"] = cap_MW
            df_resource.loc[index, "Max_Cap_MW"] = cap_MW
        
            if "Max_Cap_MWh" in df_resource.columns:
                cap_MWh = capacity_to_enforce[capacity_to_enforce["Resource"] == row["Resource"]]["EndEnergyCap"]
                cap_MWh = cap_MWh.iloc[0] if not cap_MWh.empty else None  # Extract scalar or set to None

                df_resource.loc[index, "Max_Cap_MWh"] = cap_MWh
                df_resource.loc[index, "Min_Cap_MWh"] = cap_MWh


    dataframes_dict["df_storage"]
    # Save new files
    if os.path.exists(path_storage):
        dataframes_dict["df_storage"].to_csv(path_storage, index=False)

    if os.path.exists(path_thermal):
        dataframes_dict["df_thermal"].to_csv(path_thermal, index=False)

    if os.path.exists(path_hydro):
        dataframes_dict["df_hydro"].to_csv(path_hydro, index=False)

    if os.path.exists(path_electrolyzer):
        dataframes_dict["df_electrolyzer"].to_csv(path_electrolyzer, index=False)

    if os.path.exists(path_vre):
        dataframes_dict["df_vre"].to_csv(path_vre, index=False)

    if os.path.exists(path_Vre_stor):
        dataframes_dict["df_vre_storage"].to_csv(path_Vre_stor, index=False)

In [149]:
create_resource_policies_enforce_cap(current_working_direct, path_results_to_enforce, source_name)